* Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence
from mol2vec.helpers import IdentifierTable

* Sanofi provided us the dataset which includes different information about the molecules. The feature in our dataset are : Molecular Weight, Number of Rings, Minimum Degree, Smiles, Measured log solubility in mols per litre. We should preprocess the column Smiles which defines the chemical structure of the molecules.

In [ ]:
df = pd.read_csv('../data/raw/esol.csv')
print(df.shape)
df.head()

* Convert smiles into RdKit mols and generate a list of identifiers for each molecule using the function mol2vec.mol2alt_sentence. We map each substructure to a particular identifier.

In [ ]:
def smiles_to_mols(df):
    return [Chem.MolFromSmiles(x) for x in list(df['smiles'])]

def mols_to_sentences(df):
    return [mol2alt_sentence(mol, 1) for mol in mols]

mols = smiles_to_mols(df)
mol_sentences = mols_to_sentences(df)
substructure_identifiers = {id for mol in mol_sentences for id in mol}

print(f'Nr of Molecules : {len(mols)}')
print(f'Nr of Unique Substructures : {len(substructure_identifiers)}')

* Plot the structure of the first n molecules to get a first overview and better understand the structure of our molecules.

In [ ]:
Draw.MolsToGridImage(mols[:1], molsPerRow=5, useSVG=False)

* Plot the substructures and their corresponding identifier for a particular molecule
* To depict all identifiers one can use IdentifierTable object -IdentifierTable(identifiers_to_depict, mols_containing_selected_identifiers, sentences_for_mols, number_of_columns, radius)

In [ ]:
# lets plot the substructures and their corresponding identifiers of the first molecule
IdentifierTable(mol_sentences[0][:9], mols, mol_sentences, 3, 1)

* Mol2vec is based on Word2vec algorithm and we first have to encode molecules as sentences, meaning that each substructure (represented by Morgan identifier) represents a word. 
* Load a pre-trained Mol2vec model which was trained on 20 million compounds downloaded from ZINC using: radius 1, UNK to replace all identifiers that appear less than 4 times, skip-gram and window size of 10 (The window size controls the size of the context), resulting in 100 dimensional embeddings, 
* [ZINC](https://pubchem.ncbi.nlm.nih.gov/source/ZINC) is a free database of commercially-available compounds for virtual screening. ZINC contains tens of millions of purchasable compounds in ready-to-dock, 3D formats. ZINC is provided by the Irwin and Shoichet Laboratories in the Department of Pharmaceutical Chemistry at the University of California, San Francisco (UCSF).
* An identifier is a unique number used to represent a substructure (word) of a molecule (sentence). The pre-trained Mol2vec model includes the feature vectors (representation) of 21003 substructures (vocabulary size).
* The skip-gram model is a simple neural network with one hidden layer trained in order to predict the probability of a given word being present when an input word is present. In this architecture, it takes the current word as an input and tries to accurately predict the words before and after this current word. 
* Mol2vec pre-trained model has a dictionary where the key is the identifier (substructure) and the value is the corresponding feature vector.

In [ ]:
model = word2vec.Word2Vec.load("../models/model_100dim.pkl")
# vocabulary size (nr of substructures that are represented by a feature vector in our pretrained model)
len(model.wv.key_to_index.keys())


* Plot the distribution (histogram) of the number of substructures among all molecules. 

In [ ]:
num_substructures = [len(sent) for sent in mol_sentences]
plt.figure(figsize=(10, 8))
sns.histplot(num_substructures)
plt.xlabel('Number of Substructures in Molecule')
plt.ylabel('Frequency')
plt.show()

* As a molecule is a sentence (list of identifiers) we get a list of feature vectors for each molecule.
* To get a single vector representation per molecule we compute the mean of the feature vectors of the molecule's substructures. We can then use these feature vectors to build machine learning models for supervised or unsupervised tasks.
* To perform the aforementioned steps, we implemented a function: For details see `feature_vectors.py`

In [ ]:
import sys
sys.path.append("..")
from src.features.smiles_to_feature_vector import generate_feature_vectors_from_smiles
generate_feature_vectors_from_smiles(df.loc[42:42, :]) # convert one molecule

* The Mol2Vec pretrained model does not have the vector representation for some identifiers. Let's analyse how often the problem occurs in our dataset:

In [ ]:
def count_num_available(sentence, model):
    return len([id for id in sentence if id in model.wv])

num_available = [count_num_available(sent, model) for sent in mol_sentences]
ratio_available = [avail / all for avail, all in zip(num_available, num_substructures)]

all_ids_available = [avail == all for avail, all in zip(num_available, num_substructures)]
num_incomplete_sentences = len(mol_sentences) - sum(all_ids_available)
print("Number of molecules that miss at least one identifier: " + str(num_incomplete_sentences))
print("This is " + str(num_incomplete_sentences / len(mol_sentences) * 100) + " % of the dataset")


* Around 97% of molecules are completely contained in our pretrained model
* We can also plot the frequency of the existing/available identifiers (substructures) for all molecules using a histogram, to analyse the remaining 3%:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13,4))
sns.histplot(ax=axes[0], x=[n for n, all_avail in zip(num_available, all_ids_available) if not all_avail], bins=40)
axes[0].set_title(f'Absolute number of available substructures\n for molecules with missing ids')

sns.histplot(ax=axes[1], x=[n for n, all_avail in zip(ratio_available, all_ids_available) if not all_avail], bins=50)
axes[1].set_title(f'Ratio of available substructures\n for molecules with missing ids')

* One molecule (check out boxplot) has only 1 identifier which has no available vector representation in our model. Therefore the relative frequency is equal to 0.  
* Since for some molecules, many identifiers are missing, we would end up using not a good numerical representation of the molecule and it may lead to bad results in the model performance. So lets remove molecules that have this relative frequency lower than some specified threshold.
* We can set this threshold with the parameter `existing_ids_threshold` of our function. The default is 1.0 which means all molecules with missing ids are removed

In [ ]:
vfs = generate_feature_vectors_from_smiles(df.loc[10:20, :], existing_ids_threshold=1.00)
print("Indices of molecules for which feature vectors were generated (threshold = 100%):")
print(vfs.keys())

vfs = generate_feature_vectors_from_smiles(df.loc[10:20, :], existing_ids_threshold=0.85)
print("Indices of molecules for which feature vectors were generated (threshold = 85%):")
print(vfs.keys())